In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm_notebook
import datetime
import time
import random
from joblib import Parallel, delayed


import lightgbm as lgb
from tensorflow import keras
from gplearn.genetic import SymbolicRegressor
from catboost import Pool, CatBoostRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.feature_selection import RFECV, SelectFromModel

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [3]:
train_X_0 = pd.read_csv("../input/lanl-masters-features-creating-0/train_X_features_865.csv")
train_X_1 = pd.read_csv("../input/lanl-masters-features-creating-1/train_X_features_865.csv")
y_0 = pd.read_csv("../input/lanl-masters-features-creating-0/train_y.csv", index_col=False,  header=None)
y_1 = pd.read_csv("../input/lanl-masters-features-creating-1/train_y.csv", index_col=False,  header=None)

In [4]:
train_X = pd.concat([train_X_0, train_X_1], axis=0)
train_X = train_X.reset_index(drop=True)
print(train_X.shape)
train_X.head()

(33000, 865)


,FFT_Mag_01q0,FFT_Mag_10q0,FFT_Mag_90q0,FFT_Mag_99q0,FFT_Mag_mean0,FFT_Mag_std0,FFT_Mag_max0,FFT_Phz_mean0,FFT_Phz_std0,FFT_Mag_01q2500,...,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,135.836711,421.371206,3837.759402,6112.131639,1739.925798,1385.814449,9064.175486,-0.000510,0.910909,417.854520,...,0.211731,5.464,3.769,4.301,4.401,5.116,5.262,1.409396e-06,74648.999921,5.464
1,124.732839,392.456548,2250.212819,3824.489952,1231.293809,828.294402,9080.893195,0.023934,0.915573,305.137857,...,0.248184,5.801,4.243,4.506,4.685,5.498,5.659,7.442953e-06,74397.510027,5.801
2,106.434758,369.227304,1825.186779,3035.388361,1043.817464,661.241082,8666.843710,-0.007041,0.904811,207.531656,...,0.217408,4.764,3.526,3.702,3.786,4.507,4.604,8.389262e-07,74227.207542,4.764
3,120.608863,392.794334,2468.215191,4139.927445,1271.285220,873.069679,9043.242078,-0.018745,0.904419,258.196842,...,0.244447,5.302,2.819,3.206,3.418,4.248,4.354,1.046980e-06,74415.594773,5.302
4,132.265220,444.217367,5397.274335,8740.417592,2212.343813,2080.628283,20996.324962,-0.017078,0.912532,660.563845,...,0.284440,5.657,3.187,3.872,4.037,4.953,5.190,4.496644e-07,74234.314526,5.657


In [5]:
y = pd.concat([y_0, y_1], axis=0)
y = y.reset_index(drop=True)
y[0].shape

(33000,)

In [20]:
print(train_X.shape)
print(y.shape)
print(test_X.shape)

(33000, 865)
(33000, 1)
(2624, 865)


In [5]:
train_y = pd.Series(y[0].values)

In [19]:
test_X = pd.read_csv("../input/lanl-masters-features-creating-0/test_X_features_10.csv")
del test_X["seg_id"]

In [10]:
scaler = StandardScaler()
train_columns = train_X.columns

train_X[train_columns] = scaler.fit_transform(train_X[train_columns])
test_X[train_columns] = scaler.transform(test_X[train_columns])

In [8]:
# cols = train_X.columns

# df = pd.concat([train_X, test_X[cols]], axis=0)
# df = df.reset_index(drop=True)
# df[cols] = np.round(df.values, 3)

# for col in cols:
#     df[col+"_count"] = df[col].map(df[col].value_counts())
    
# count_cols = [i for i in df.columns if "_count" in i]

# train_X = pd.concat([train_X, df.loc[:40000-1, count_cols]], axis=1)
# test_X = pd.concat([test_X, df.loc[40000:, count_cols].reset_index(drop=True)], axis=1)
# train_X.shape, test_X.shape

# CatBoost

In [9]:
train_columns = train_X.columns
n_fold = 5

In [10]:
%%time
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

oof = np.zeros(len(train_X))
train_score = []
fold_idxs = []
# if PREDICTION: 
predictions = np.zeros(len(test_X))

feature_importance_df = pd.DataFrame()
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    fold_idxs.append(val_idx)

    X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]

    model = CatBoostRegressor(n_estimators=25000, verbose=-1, objective="MAE", loss_function="MAE", boosting_type="Ordered", task_type="GPU")
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_val, y_val)], 
#               eval_metric='mae',
              verbose=2500, 
              early_stopping_rounds=500)
    oof[val_idx] = model.predict(X_val)

    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = model.feature_importances_[:len(train_columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions
#     if PREDICTION:

    predictions += model.predict(test_X[train_columns]) / folds.n_splits
    train_score.append(model.best_score_['learn']["MAE"])

cv_score = mean_absolute_error(train_y, oof)
print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

fold 0
0:	learn: 5.6534251	test: 5.7103149	best: 5.7103149 (0)	total: 67.4ms	remaining: 28m 5s
2500:	learn: 1.6530211	test: 1.7309088	best: 1.7309088 (2500)	total: 2m 36s	remaining: 23m 25s
5000:	learn: 1.4259490	test: 1.5684751	best: 1.5684751 (5000)	total: 5m 15s	remaining: 21m
7500:	learn: 1.2678242	test: 1.4671433	best: 1.4671433 (7500)	total: 7m 56s	remaining: 18m 31s
10000:	learn: 1.1491418	test: 1.3998904	best: 1.3998904 (10000)	total: 10m 38s	remaining: 15m 56s
12500:	learn: 1.0529444	test: 1.3500095	best: 1.3500095 (12500)	total: 13m 19s	remaining: 13m 19s
15000:	learn: 0.9738277	test: 1.3117460	best: 1.3117415 (14999)	total: 16m 1s	remaining: 10m 40s
17500:	learn: 0.9049903	test: 1.2806047	best: 1.2806043 (17499)	total: 18m 44s	remaining: 8m 1s
20000:	learn: 0.8455384	test: 1.2553391	best: 1.2553378 (19998)	total: 21m 26s	remaining: 5m 21s
22500:	learn: 0.7927802	test: 1.2344988	best: 1.2344988 (22500)	total: 24m 9s	remaining: 2m 41s
24999:	learn: 0.7474378	test: 1.2174123	be

In [11]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = predictions
submission.to_csv(f'CatBoost_{today}_test_{cv_score:.3f}_train_{np.mean(train_score):.3f}.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,2.806046
1,seg_0012b5,5.183179
2,seg_00184e,4.890522
3,seg_003339,8.580695
4,seg_0042cc,6.781291


# Neural Network

In [12]:
def create_model(input_dim=10):

    model = keras.Sequential()
    model.add(keras.layers.Dense(256, activation="relu", input_dim=input_dim))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(96, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1, activation="linear"))

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #'rmsprop'
    model.compile(optimizer=optimizer,loss='mae')
    return model

patience = 50
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

In [13]:
%%time
# n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

NN_oof = np.zeros(len(train_X))
train_score = []
fold_idxs = []

NN_predictions = np.zeros(len(test_X))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    fold_idxs.append(val_idx)
    
    X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    model = create_model(train_X.shape[-1])
    model.fit(X_tr, y_tr, epochs=500, batch_size=32, verbose=0, callbacks=[call_ES,], validation_data=[X_val, y_val]) #
    
    NN_oof[val_idx] = model.predict(X_val)[:,0]
    
    NN_predictions += model.predict(test_X[train_columns])[:,0] / folds.n_splits
    history = model.history.history
    tr_loss = history["loss"]
    val_loss = history["val_loss"]
    print(f"loss: {tr_loss[-patience]:.3f} | val_loss: {val_loss[-patience]:.3f} | diff: {val_loss[-patience]-tr_loss[-patience]:.3f}")
    train_score.append(tr_loss[-patience])
#     break
    
cv_score = mean_absolute_error(train_y, NN_oof)
print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

fold 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Restoring model weights from the end of the best epoch.
Epoch 00287: early stopping
loss: 0.884 | val_loss: 1.382 | diff: 0.498
fold 1
Restoring model weights from the end of the best epoch.
Epoch 00428: early stopping
loss: 0.816 | val_loss: 1.401 | diff: 0.586
fold 2
Restoring model weights from the end of the best epoch.
Epoch 00284: early stopping
loss: 0.910 | val_loss: 1.459 | diff: 0.549
fold 3
Restoring model weights from the end of the best epoch.
Epoch 00279: early stopping
loss: 0.897 | val_loss: 1.457 | diff: 0.560
fold 4
Restoring model weights from the end of the best epoch.
Epoch 00424: early stopping
loss: 0.815 | val_loss: 1.420 | diff: 0.604
After 5 test_CV = 1.339 | train_CV = 0.864 | 0.475 CPU times: user 2h 29min 31s, sys: 12mi

In [14]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = NN_predictions
submission.to_csv(f'NN_{today}_test_{cv_score:.3f}_train_{np.mean(train_score):.3f}.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,2.612691
1,seg_0012b5,4.810834
2,seg_00184e,3.973062
3,seg_003339,7.660580
4,seg_0042cc,6.638792


# GPLearn

In [15]:
# best_parameters = {'tournament_size': 17, 'population_size': 4000, 'p_crossover': 0.8, 'generations': 18}
# function_set = ('add', 'sub', 'mul', 'div', "sqrt", "log", "max", "min", "sin", "cos", "tan")

In [16]:
# %%time
# # n_fold = 5
# folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

# GPL_oof = np.zeros(len(train_X))
# GPL_predictions = np.zeros(len(test_X))
# train_score = []

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
#     strLog = "fold {}".format(fold_)
#     print(strLog)
    
#     X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
#     y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    
#     model = SymbolicRegressor(**best_parameters, stopping_criteria=0.0,const_range=(-1.0, 1.0), init_depth=(2, 6), init_method='half and half', 
#                           function_set=function_set, metric='mean absolute error', parsimony_coefficient=0.001,
#                           p_subtree_mutation=0.01, p_hoist_mutation=0.01, p_point_mutation=0.01, 
#                           p_point_replace=0.05, max_samples=1.0, feature_names=None, 
#                           warm_start=False, low_memory=False, n_jobs=-1, verbose=1, random_state=42)
    
#     model.fit(X_tr, y_tr) #
    
#     GPL_oof[val_idx] = model.predict(X_val)
#     GPL_predictions += model.predict(test_X[train_columns]) / folds.n_splits
    
#     train_score.append(model.run_details_["best_fitness"][-1])
# #     break
    
# cv_score = mean_absolute_error(train_y, GPL_oof)
# print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

In [17]:
# today = str(datetime.date.today())
# submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

# submission["time_to_failure"] = GPL_predictions
# submission.to_csv(f'GPL_{today}_test_{cv_score:.3f}_train_{np.mean(train_score):.3f}.csv', index=False)
# submission.head()

# Final Submission

In [18]:
Scirpus_prediction = pd.read_csv("../input/andrews-new-script-plus-a-genetic-program-model/gpI.csv")
Scirpus_prediction.head()

,seg_id,time_to_failure
0,seg_00030f,2.856751
1,seg_0012b5,4.189392
2,seg_00184e,5.538267
3,seg_003339,9.051419
4,seg_0042cc,7.099581


In [19]:
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = (predictions+NN_predictions+Scirpus_prediction.time_to_failure.values)/3
submission.to_csv(f'FINAL_{today}_submission.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,2.758496
1,seg_0012b5,4.727802
2,seg_00184e,4.800617
3,seg_003339,8.430898
4,seg_0042cc,6.839888
